# 아이유팬이 좋아할 만한 다른 아티스트 찾기

In [1]:
# 모듈 import
import pandas as pd
import os
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

In [2]:
# 파일 불러오기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 counts로 변경
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 사용하는 컬럼 재정의
using_cols = ['user_id', 'movie_id', 'counts']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [7]:
# 메타데이터 확인
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [8]:
# idx_to_movie
idx_to_movie = {}
for i in range(len(movies)):
    idx_to_movie[movies['movie_id'][i]] = movies['title'][i]

In [9]:
# movie_to_idx
movie_to_idx = {}
for k, v in idx_to_movie.items():
    movie_to_idx[v] = k

In [10]:
# 나의 추천 목록 생성
my_favorite = [2, 10, 247, 316, 345]
my_movielist = pd.DataFrame({'user_id':[6041]*5, 'movie_id':my_favorite, 'counts':[5, 4, 4, 3, 5]})

my_movielist

In [13]:
if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movielist)

In [14]:
# 유저 수
ratings['user_id'].nunique()

6040

In [15]:
# 영화 수
ratings['movie_id'].nunique()

3628

In [16]:
# 인기 많은 영화
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [17]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6040.000000
mean      138.490563
std       156.238108
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [25]:
# csr_data의 출력 차원
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

print(num_user, num_movie)

6040 3628


In [26]:
# csr_data 생성
csr_data = csr_matrix((ratings['counts'], (ratings['user_id'], ratings['movie_id'])))
csr_data # shape가 일치하지 않아서 ValueError가 발생하였기 때문에 shape parameter를 비워두었다.

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [28]:
# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [30]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [31]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [32]:
# 벡터값 확인
my_vector, movie_vector = als_model.user_factors[6041], als_model.item_factors[2]

In [33]:
my_vector

array([ 0.2744872 ,  0.1663075 ,  0.46568924, -0.60990584, -0.02253972,
        0.53355056,  0.42762375, -0.01783347, -0.8774152 ,  0.8062551 ,
        0.3825643 , -0.43335786,  0.17930639,  0.00239477, -0.5527775 ,
       -0.08531985, -0.04650709, -0.37197092,  0.25369734, -0.1967742 ,
        0.35300133,  0.06819372,  0.6953232 , -0.39468843, -0.10923447,
       -0.31646132, -0.62653965,  0.34147188,  0.47767663,  0.35947862,
        0.2306574 ,  0.3601222 , -0.19829217, -0.13011886,  0.4401776 ,
       -0.08786922, -0.35846603,  0.06507821,  0.3952418 ,  0.28217632,
        0.22063032, -0.1493413 , -0.32126355, -0.25919467, -0.68212724,
        0.1368915 ,  0.53166807, -0.06495571, -0.6725214 ,  0.19055149,
        0.19620568, -0.6308462 , -0.07425918,  0.39590588,  0.6604617 ,
        0.05835024, -0.1537444 , -0.84491926,  0.62593174,  0.2517935 ,
       -0.54200697, -0.14326458,  0.17045455, -0.4313797 ,  0.21857974,
       -1.078127  ,  0.5377187 , -0.283514  ,  0.33317053, -0.30

In [34]:
movie_vector

array([-0.00940614, -0.00907871,  0.02170591,  0.02061642, -0.01337398,
       -0.00189943,  0.0262162 , -0.00570551, -0.00491408,  0.04038746,
        0.03604221,  0.00313791,  0.00885167, -0.01709836,  0.0013188 ,
        0.01627013,  0.00317396, -0.01141351,  0.01108231, -0.00352134,
        0.03090933,  0.00662951,  0.02718726, -0.00178463,  0.00086973,
        0.02535204, -0.01037805,  0.02030583,  0.02222096,  0.0249093 ,
        0.01909581, -0.0020723 , -0.01248516,  0.01805308,  0.03587919,
        0.00766241, -0.00660969,  0.00945288,  0.00434788,  0.01303989,
       -0.00428161, -0.01329738,  0.01105775, -0.00125104, -0.00227024,
        0.02174228,  0.04276942,  0.00438238, -0.01005146,  0.0254362 ,
        0.02516734, -0.01178009,  0.00392994,  0.00755019,  0.01358285,
        0.00980846, -0.00519975, -0.00046347,  0.01326628,  0.01158505,
       -0.00269135,  0.01107055,  0.01967713, -0.0019084 ,  0.00604885,
       -0.02132984,  0.0032553 , -0.00066832,  0.01641535,  0.00

In [35]:
# 내적
np.dot(my_vector, movie_vector)

0.33719453

In [36]:
# 비슷한 영화 찾기
favorite_movie_id = 1
similar_movie = als_model.similar_items(favorite_movie_id, N=15)
similar_movie

[(1, 0.9999999),
 (3114, 0.80435133),
 (2355, 0.59794253),
 (588, 0.59449553),
 (1265, 0.53959656),
 (34, 0.5347627),
 (364, 0.4931068),
 (2321, 0.4644613),
 (595, 0.45921063),
 (1923, 0.43079963),
 (2396, 0.38387454),
 (1907, 0.37349546),
 (317, 0.36247477),
 (367, 0.35824013),
 (356, 0.35753328)]

In [37]:
[idx_to_movie[i[0]] for i in similar_movie]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Babe (1995)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)",
 'Shakespeare in Love (1998)',
 'Mulan (1998)',
 'Santa Clause, The (1994)',
 'Mask, The (1994)',
 'Forrest Gump (1994)']

In [38]:
# 비슷한 아티스트를 찾아주는 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [39]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Babe (1995)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)"]

In [40]:
# 비슷한 아티스트를 찾아주는 함수
def get_similar_movie_id(movie_id: str):
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [41]:
# Toy Story와 비슷한 영화 추천
get_similar_movie_id(1)

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Babe (1995)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)"]

## 회고록

- csr_data를 만드는데 왜인지 모르겠지만 unique로 뽑은 값을 shape에다 넣어줬더니 row index가 넘었다고 에러가 떴다. 그래서 shape를 넣지 않고 그냥 돌렸더니 생성이 되었고, 만들어진 csr_data의 shape를 확인해보니 unique값과 달랐다. 왜 그런지는 잘 모르겠다... NaN값이 있나..?
- 오늘 한 과제가 지금까지 한 과제 중에서 가장 어려웠던 것 같다. 아직 데이터 전처리에 익숙하지 않아서 그럴 수도 있지만 데이터 전처리가 거의 전부인 것 같다.
- 데이터 전처리만 잘 해도 반은 성공한다는 느낌이다. 애초에 데이터가 없으면 시작조차 할 수 없으니 데이터 전처리하는 방법과 pandas, numpy등 사용 방법에 대해 잘 익혀둬야 겠다.
- 모델을 훈련하여 실제로 추천받은 목록을 보니 토이스토리를 골랐을 때 토이스토리2, 벅스라이프, 알라딘 등을 추천해주는 걸 보면 만화애니메이션 장르쪽을 추천해주고 있다. 따라서 훈련이 잘 이루어졌다고 평가할 수 있을 것 같다.
- csr_data를 만들 때 shape 에서 Error가 발생하는 이유를 알아냈다. 그 원인은 csr_data의 (row_ind, col_ind) parameter가 max(row_ind), max(col_ind)로 작동하여 row_ind와 col_ind의 index의 최댓값을 사용하기 때문이다. 물론 row와 col이 index 순으로 잘 정렬되어 있다면 이렇게 해도 문제가 없지만, 실제로는 movie_id의 중간에 빠진 index들이 있기 때문에 movie_id의 총 갯수인 3628개 보다 큰 max(row_ind)의 3953개가 parameter로 사용되는 것이다. user_id도 마찬가지로 unique한 값은 6040개지만, index가 1부터 시작하여 끝값은 6041이므로 총 6042개를 col_ind로 사용하게 된다. 이 부분은 수정해보려고 했으나, movie_id마다 이미 할당된 title이 있기 때문에 ratings DataFrame에 다시 movie_id에 맞는 title column을 더해주고 movie_id순으로 중복을 제거하고 sort하여 title에 다시 movie_id를 할당해 주는 작업이 너무 번거로워서 그만뒀다.